In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import csv
import pandas as pd
from scipy import stats
import re
from scipy.stats.stats import pearsonr
#import rpy2.robjects as robjects
import random
from statsmodels.stats.multitest import fdrcorrection
import os
import seaborn as sns
#import gseapy as gs
from scipy.stats import norm
import gseapy as gs

min_gen = 6
max_gen = 300

#List of imprinted genes was obtained from https://www.geneimprint.com/site/genes-by-species
imp = pd.read_csv("Imprinted_Genes.csv")
imp_conf = imp[imp["Status"] == "Imprinted"]
imp_conf_list = list(imp_conf["Gene"])

In [2]:
#Read in HPO.
HPO = pd.read_csv("HPO_World_14_Typ.csv")
HPO = HPO.set_index("HPO_IDs")
HPO_Conv = pd.read_csv("HPO_Conversion.csv")
Convert_HPO = {}
gene_set_hpo_brain = []
gene_set_hpo_skel = []
for index, row in HPO_Conv.iterrows():
    Convert_HPO[row[0]] = row[1]
HPO.head()

d_HPO = {}
for index, row in HPO.iterrows():
    x = Convert_HPO[index]
    d_HPO[x] = []
    for i in zip(row.index.tolist(), list(row)):
        if i[1]:
            d_HPO[Convert_HPO[index]].append(i[0])

phen = pd.read_csv("specific_phenotype_ontology_v14.csv")
keeper1 = "brain"
keep_brain = []
keeper2 = "skel"
keep_skel = []
for index, row in phen.iterrows():
    k = row[4]
    j = row[3]
    try:
        if keeper1 in k:
            keep_brain.append(row[1])
    except:
        pass
    try:
        if keeper2 in j:
            keep_skel.append(row[1])
    except:
        pass
d_HPO_brain = {}
d_HPO_skel = {}
for key in d_HPO.keys():
    if key in keep_brain:
        d_HPO_brain[key] = d_HPO[key]
        if len(d_HPO_brain[key]) >= min_gen and len(d_HPO_brain[key]) <= max_gen:
            gene_set_hpo_brain = gene_set_hpo_brain + d_HPO_brain[key]
    if key in keep_skel:
        d_HPO_skel[key] = d_HPO[key]
        if len(d_HPO_skel[key]) >= min_gen and len(d_HPO_skel[key]) <= max_gen:
            gene_set_hpo_skel = gene_set_hpo_skel + d_HPO_skel[key]
        
            
gene_set_hpo_brain = list(set(gene_set_hpo_brain))
print(len(gene_set_hpo_brain))
gene_set_hpo_skel = list(set(gene_set_hpo_skel))


3394


In [3]:
#Read in Gene Ontology
#Note that GOBP was not used as it is often redundant with other categories and crashes my computer as well as computers on the cluster.
d_CC = {}
d_BP = {}
d_MF = {}
gene_set_cc = []
gene_set_bp = []
gene_set_mf = []
CC = pd.read_csv("OFFICIAL_GENE_SYMBOL2GOTERM_CC_DIRECT.txt", sep = "\t", header = None)
for index, row in CC.iterrows():
    if row[1] not in d_CC.keys():
        d_CC[row[1]] = [row[0]]
    else:
        d_CC[row[1]].append(row[0])
        
BP = pd.read_csv("OFFICIAL_GENE_SYMBOL2GOTERM_BP_DIRECT.txt", sep = "\t", header = None)
for index, row in BP.iterrows():
    if row[1] not in d_BP.keys():
        d_BP[row[1]] = [row[0]]
    else:
        d_BP[row[1]].append(row[0])
        
MF = pd.read_csv("OFFICIAL_GENE_SYMBOL2GOTERM_MF_DIRECT.txt", sep = "\t", header = None)
for index, row in MF.iterrows():
    if row[1] not in d_MF.keys():
        d_MF[row[1]] = [row[0]]
    else:
        d_MF[row[1]].append(row[0])

for key in d_MF.keys():
    z = d_MF[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_mf = gene_set_mf + z
for key in d_CC.keys():
    z = d_CC[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_cc = gene_set_cc + z
for key in d_BP.keys():
    z = d_BP[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_bp = gene_set_bp + z
gene_set_cc = list(set(gene_set_cc))
gene_set_bp = list(set(gene_set_bp))
gene_set_mf = list(set(gene_set_mf))

In [4]:
#Read in GAD
d_GAD = {}
d_GADC = {}
gene_set_gad = []
GAD = pd.read_csv("OFFICIAL_GENE_SYMBOL2GAD_DISEASE.txt", sep = "\t", header = None)
GADC = pd.read_csv("OFFICIAL_GENE_SYMBOL2GAD_DISEASE_CLASS.txt", sep = "\t", header = None)
for index, row in GAD.iterrows():
    if row[1] not in d_GAD.keys():
        d_GAD[row[1]] = [row[0]]
    else:
        d_GAD[row[1]].append(row[0])
for index, row in GADC.iterrows():
    if row[1] not in d_GADC.keys():
        d_GADC[row[1]] = [row[0]]
    else:
        d_GADC[row[1]].append(row[0])
gene_set_GAD = []
for key in d_GAD.keys():
    z = d_GAD[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_GAD = gene_set_GAD + z
gene_set_GAD = list(set(gene_set_gad))

In [5]:
#Read in KEGG
d_KEGG = {}
gene_set_kegg = []
KEGG = pd.read_csv("KEGG_pathway_NEW.txt", sep = "\t")
for index, row in KEGG.iterrows():
    if row["pathway name"] not in d_KEGG.keys():
        d_KEGG[row["pathway name"]] = [row["gene symbol"]]
    else:
        d_KEGG[row["pathway name"]].append(row["gene symbol"])
        
for key in d_KEGG.keys():
    z = d_KEGG[key]
    if len(z) >= min_gen and len(z) <= max_gen:
        gene_set_kegg = gene_set_kegg + z
gene_set_kegg = list(set(gene_set_kegg))

In [6]:
#Finally read in REACTOME
d_REACT = {}
gene_set_react = []
REACT = pd.read_csv("OFFICIAL_GENE_SYMBOL2REACTOME_PATHWAY.txt", sep = "\t", header = None)
for index, row in REACT.iterrows():
    gene_set_react.append(row[0])
    if row[1] not in d_REACT:
        d_REACT[row[1]] = [row[0]]
    else:
        d_REACT[row[1]].append(row[0])
gene_set_react = list(set(gene_set_react))

In [ ]:
####### USED

In [ ]:
def read_in_for_MWU_GSEA_rank(i, j):
    f = pd.read_csv(i + "_Ranks_MinReads5_" + j + ".csv")
    f = f.sort_values(by="MWU p-val Rank", ascending=False)

    f = f[~f["gene"].isin(imp_conf_list)]
    l = list(f.columns)
    l[0] = "gene"
    f.columns = l
    f = pd.DataFrame(f[["gene", "MWU p-val Rank"]])
    f.columns = [0, 1]
    f[1] = f[1]
    return f
db = [d_KEGG, d_CC, d_MF, d_REACT, d_HPO_brain]
#db = [d_KEGG, d_CC, d_BP, d_MF, d_REACT, d_HPO_skel]
cells = ["D50", "D100", "D150"]
dat = ["GTEx"]
for cell in cells:
    for datum in dat:
        test = read_in_for_MWU_GSEA_rank(cell, datum)
        for j in range(len(db)):
            data = db[j]
            if j == 0:
                out_name_1 = datum + "_" + cell + "_MinReads5_pval_ranks_gsea_Enrich_KEGG"
                gene_set = gene_set_kegg
            elif j == 1:
                out_name_1 = datum + "_" + cell + "_MinReads5_pval_ranks_gsea_Enrich_GOCC"
                gene_set = gene_set_cc
            elif j == 22:
                out_name_1 = datum + "_" + cell + "_MinReads5_pval_ranks_gsea_Enrich_GOBP"
                gene_set = gene_set_bp
            elif j == 2:
                out_name_1 = datum + "_" + cell + "_MinReads5_pval_ranks_gsea_Enrich_GOMF"
                gene_set = gene_set_mf
            elif j == 3:
                out_name_1 = datum + "_" + cell + "_MinReads5_pval_ranks_gsea_Enrich_REACT"
                gene_set = gene_set_react
            elif j == 4:
                out_name_1 = datum + "_" + cell + "_MinReads5_pval_ranks_gsea_Enrich_HPO_Brain"
                gene_set = gene_set_hpo_brain
            elif j == 17:
                if cell == "iPSC" or cell == "CNCC":
                    out_name_1 = datum + "_" + cell + "_DESeq2_l2fc_l2fc_diff_gsea_Enrich_HPO_Skel"
                    gene_set = gene_set_hpo_skel
                else:
                    out_name_1 = datum + "_" + cell + "_DESeq2_l2fc_l2fc_diff_gsea_Enrich_HPO_Brain"
                    gene_set = gene_set_hpo_brain
            count = 0
            for ranking in [test]:
                if count:
                    out_name = out_name_1 + ".csv"
                else:
                    out_name = out_name_1 + ".csv"
                print(cell, datum, j)
                gs.prerank(rnk=ranking, gene_sets=data, processes=4, permutation_num=1000, outdir= 'C:/Users/astar/Z-score/' + out_name_1, format='png', seed=6, min_size = 10, max_size = 300)
                count += 1

In [7]:
def read_in_for_MWU_GSEA_rank(i, j):
    f = pd.read_csv(i + "_Ranks_p05_" + j + ".csv")
    f = f.sort_values(by="MWU p-val DESeq2 p-val Rank Rank Dif", ascending=False)

    f = f[~f["gene"].isin(imp_conf_list)]
    l = list(f.columns)
    l[0] = "gene"
    f.columns = l
    f = pd.DataFrame(f[["gene", "MWU p-val DESeq2 p-val Rank Rank Dif"]])
    f.columns = [0, 1]
    f[1] = f[1]
    return f
#db = [d_BP]
db = [d_KEGG, d_CC, d_MF, d_REACT, d_HPO_brain]
cells = ["D50", "D100", "D150"]
dat = ["GTEX"]
for cell in cells:
    for datum in dat:
        test = read_in_for_MWU_GSEA_rank(cell, datum)
        for j in range(len(db)):
            data = db[j]
            if j == 0:
                out_name_1 = datum + "_" + cell + "_p05_p-val_DESeq2_p-val_Rank_Rank_Dif_gsea_Enrich_KEGG"
                gene_set = gene_set_kegg
            elif j == 1:
                out_name_1 = datum + "_" + cell + "_p05_p-val_DESeq2_p-val_Rank_Rank_Dif_gsea_Enrich_GOCC"
                gene_set = gene_set_cc
            elif j == 22:
                out_name_1 = datum + "_" + cell + "_p05_p-val_DESeq2_p-val_Rank_Rank_Dif_gsea_Enrich_GOBP"
                gene_set = gene_set_bp
            elif j == 2:
                out_name_1 = datum + "_" + cell + "_p05_p-val_DESeq2_p-val_Rank_Rank_Dif_gsea_Enrich_GOMF"
                gene_set = gene_set_mf
            elif j == 3:
                out_name_1 = datum + "_" + cell + "_p05_p-val_DESeq2_p-val_Rank_Rank_Dif_gsea_Enrich_REACT"
                gene_set = gene_set_react
            elif j == 4:
                out_name_1 = datum + "_" + cell + "_p05_p-val_DESeq2_p-val_Rank_Rank_Dif_gsea_Enrich_HPO_Brain"
                gene_set = gene_set_hpo_brain
            elif j == 17:
                if cell == "iPSC" or cell == "CNCC":
                    out_name_1 = datum + "_" + cell + "_DESeq2_l2fc_l2fc_diff_gsea_Enrich_HPO_Skel"
                    gene_set = gene_set_hpo_skel
                else:
                    out_name_1 = datum + "_" + cell + "_DESeq2_l2fc_l2fc_diff_gsea_Enrich_HPO_Brain"
                    gene_set = gene_set_hpo_brain
            count = 0
            for ranking in [test]:
                if count:
                    out_name = out_name_1 + ".csv"
                else:
                    out_name = out_name_1 + ".csv"
                print(cell, datum, j)
                try:
                    gs.prerank(rnk=ranking, gene_sets=data, processes=4, permutation_num=1000, outdir= 'C:/Users/astar/Z-score/' + out_name_1, format='png', seed=6, min_size = 10, max_size = 300)
                except:
                    pass
                count += 1

D50 GTEX 0
D50 GTEX 1
D50 GTEX 2
D50 GTEX 3
D50 GTEX 4
D100 GTEX 0
D100 GTEX 1
D100 GTEX 2
D100 GTEX 3
D100 GTEX 4
D150 GTEX 0
D150 GTEX 1
D150 GTEX 2
D150 GTEX 3
D150 GTEX 4


In [44]:
def read_in_for_MWU_GSEA_rank(i, j):
    f = pd.read_csv(i + "_Ranks_" + j + ".csv")
    f = f.sort_values(by="MWU p-val L2FC Rank Rank Dif", ascending=False)

    f = f[~f["gene"].isin(imp_conf_list)]
    l = list(f.columns)
    l[0] = "gene"
    f.columns = l
    f = pd.DataFrame(f[["gene", "MWU p-val L2FC Rank Rank Dif"]])
    f.columns = [0, 1]
    f[1] = f[1]
    return f
db = [d_bp, d_KEGG, d_CC, d_MF, d_REACT, d_HPO_brain]
#db = [d_KEGG, d_CC, d_BP, d_MF, d_REACT, d_HPO_skel]
cells = ["D50", "D100", "D150"]
dat = ["GTEx", "Stein"]
for cell in cells:
    for datum in dat:
        test = read_in_for_MWU_GSEA_rank(cell, datum)
        for j in range(len(db)):
            data = db[j]
            if j == 1:
                out_name_1 = datum + "_" + cell + "_MWU_p-val_L2FC_Rank_Rank_Dif_gsea_Enrich_KEGG"
                gene_set = gene_set_kegg
            elif j == 2:
                out_name_1 = datum + "_" + cell + "_MWU_p-val_L2FC_Rank_Rank_Dif_gsea_Enrich_GOCC"
                gene_set = gene_set_cc
            elif j == 0:
                out_name_1 = datum + "_" + cell + "_MWU_p-val_L2FC_Rank_Rank_Dif_gsea_Enrich_GOBP"
                gene_set = gene_set_bp
            elif j == 2:
                out_name_1 = datum + "_" + cell + "_MWU_p-val_L2FC_Rank_Rank_Dif_gsea_Enrich_GOMF"
                gene_set = gene_set_mf
            elif j == 3:
                out_name_1 = datum + "_" + cell + "_MWU_p-val_L2FC_Rank_Rank_Dif_gsea_Enrich_REACT"
                gene_set = gene_set_react
            elif j == 4:
                out_name_1 = datum + "_" + cell + "_MWU_p-val_L2FC_Rank_Rank_Dif_gsea_Enrich_HPO_Brain"
                gene_set = gene_set_hpo_brain
            elif j == 17:
                if cell == "iPSC" or cell == "CNCC":
                    out_name_1 = datum + "_" + cell + "_DESeq2_l2fc_l2fc_diff_gsea_Enrich_HPO_Skel"
                    gene_set = gene_set_hpo_skel
                else:
                    out_name_1 = datum + "_" + cell + "_DESeq2_l2fc_l2fc_diff_gsea_Enrich_HPO_Brain"
                    gene_set = gene_set_hpo_brain
            count = 0
            for ranking in [test]:
                if count:
                    out_name = out_name_1 + ".csv"
                else:
                    out_name = out_name_1 + ".csv"
                print(cell, datum, j)
                gs.prerank(rnk=ranking, gene_sets=data, processes=4, permutation_num=1000, outdir= 'C:/Users/astar/Z-score/' + out_name_1, format='png', seed=6, min_size = 10, max_size = 300)
                count += 1

D50 GTEx 0
D50 GTEx 1
D50 GTEx 2
D50 GTEx 3
D50 GTEx 4
D50 Stein 0
D50 Stein 1
D50 Stein 2
D50 Stein 3
D50 Stein 4
D100 GTEx 0
D100 GTEx 1
D100 GTEx 2
D100 GTEx 3
D100 GTEx 4
D100 Stein 0
D100 Stein 1
D100 Stein 2
D100 Stein 3
D100 Stein 4
D150 GTEx 0
D150 GTEx 1
D150 GTEx 2
D150 GTEx 3
D150 GTEx 4
D150 Stein 0
D150 Stein 1
D150 Stein 2
D150 Stein 3
D150 Stein 4


In [9]:
#Do enrichment testing on the signed ranked lists.
for k in ["MWU p-val Rank"]:
#for k in ["MWU p-val L2FC Rank Rank Dif"]:
    def read_in_for_MWU_GSEA_rank(i, j, k):
        f = pd.read_csv(i + "_Ranks_p05_" + j + "_Signed.csv")
        f = f.sort_values(by=k, ascending=False)
        f = f[~f["gene"].isin(imp_conf_list)]
        l = list(f.columns)
        l[0] = "gene"
        f.columns = l
        f = pd.DataFrame(f[["gene", k]])
        f.columns = [0, 1]
        f[1] = f[1]
        return f
    db = [d_KEGG, d_CC, d_MF, d_REACT, d_HPO_brain]
    #db = [d_KEGG, d_CC, d_BP, d_MF, d_REACT, d_HPO_skel]
    cells = ["D50", "D100", "D150"]
    dat = ["GTEx"]
    for cell in cells:
        for datum in dat:
            test = read_in_for_MWU_GSEA_rank(cell, datum, k)
            for j in range(len(db)):
                data = db[j]
                if j == 0:
                    out_name_1 = datum + "_" + cell + k.replace(" ", "_") + "_p05_Signed_gsea_Enrich_KEGG"
                    gene_set = gene_set_kegg
                elif j == 1:
                    out_name_1 = datum + "_" + cell + k.replace(" ", "_") + "_p05_Signed_gsea_Enrich_GOCC"
                    gene_set = gene_set_cc
                elif j == 22:
                    out_name_1 = datum + "_" + cell + k.replace(" ", "_") + "_p05_Signed_gsea_Enrich_GOBP"
                    gene_set = gene_set_bp
                elif j == 2:
                    out_name_1 = datum + "_" + cell + k.replace(" ", "_") + "_p05_Signed_gsea_Enrich_GOMF"
                    gene_set = gene_set_mf
                elif j == 3:
                    out_name_1 = datum + "_" + cell + k.replace(" ", "_") + "_p05_Signed_gsea_Enrich_REACT"
                    gene_set = gene_set_react
                elif j == 4:
                    out_name_1 = datum + "_" + cell + k.replace(" ", "_") + "_p05_Signed_gsea_Enrich_HPO_Brain"
                    gene_set = gene_set_hpo_brain
                count = 0
                for ranking in [test]:
                    if count:
                        out_name = out_name_1 + ".csv"
                    else:
                        out_name = out_name_1 + ".csv"
                    print(cell, datum, k)
                    gs.prerank(rnk=ranking, gene_sets=data, processes=4, permutation_num=1000, outdir= 'C:/Users/astar/Z-score/' + out_name_1, format='png', seed=6, min_size = 10, max_size = 300)
                    count += 1

D50 GTEx MWU p-val Rank
D50 GTEx MWU p-val Rank
D50 GTEx MWU p-val Rank
D50 GTEx MWU p-val Rank
D50 GTEx MWU p-val Rank
D100 GTEx MWU p-val Rank
D100 GTEx MWU p-val Rank
D100 GTEx MWU p-val Rank
D100 GTEx MWU p-val Rank
D100 GTEx MWU p-val Rank
D150 GTEx MWU p-val Rank
D150 GTEx MWU p-val Rank
D150 GTEx MWU p-val Rank
D150 GTEx MWU p-val Rank
D150 GTEx MWU p-val Rank
